In [71]:
from abc import ABC, abstractmethod
from typing import List, Self
from enum import Enum

class Visitor:
    pass

class Component(ABC):
    """
    The Component interface declares an `accept` method that should take the
    base visitor interface as an argument.
    """

    @abstractmethod
    def accept(self, visitor: Visitor) -> None:
        pass

class ConcreteComponentA(Component):
    """
    Each Concrete Component must implement the `accept` method in such a way
    that it calls the visitor's method corresponding to the component's class.
    """

    def accept(self, visitor: Visitor) -> None:
        """
        Note that we're calling `visitConcreteComponentA`, which matches the
        current class name. This way we let the visitor know the class of the
        component it works with.
        """

        visitor.visit_concrete_component_a(self)

    def exclusive_method_of_concrete_component_a(self) -> str:
        """
        Concrete Components may have special methods that don't exist in their
        base class or interface. The Visitor is still able to use these methods
        since it's aware of the component's concrete class.
        """

        return "A"


class ConcreteComponentB(Component):
    """
    Same here: visitConcreteComponentB => ConcreteComponentB
    """

    def accept(self, visitor: Visitor):
        visitor.visit_concrete_component_b(self)

    def special_method_of_concrete_component_b(self) -> str:
        return "B"

class Visitor(ABC):
    """
    The Visitor Interface declares a set of visiting methods that correspond to
    component classes. The signature of a visiting method allows the visitor to
    identify the exact class of the component that it's dealing with.
    """

    @abstractmethod
    def visit_concrete_component_a(self, element: ConcreteComponentA) -> None:
        pass

    @abstractmethod
    def visit_concrete_component_b(self, element: ConcreteComponentB) -> None:
        pass


"""
Concrete Visitors implement several versions of the same algorithm, which can
work with all concrete component classes.

You can experience the biggest benefit of the Visitor pattern when using it with
a complex object structure, such as a Composite tree. In this case, it might be
helpful to store some intermediate state of the algorithm while executing
visitor's methods over various objects of the structure.
"""


class ConcreteVisitor1(Visitor):
    def visit_concrete_component_a(self, element) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor1")

    def visit_concrete_component_b(self, element) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor1")


class ConcreteVisitor2(Visitor):
    def visit_concrete_component_a(self, element) -> None:
        print(f"{element.exclusive_method_of_concrete_component_a()} + ConcreteVisitor2")

    def visit_concrete_component_b(self, element) -> None:
        print(f"{element.special_method_of_concrete_component_b()} + ConcreteVisitor2")


def client_code(components: List[Component], visitor: Visitor) -> None:
    """
    The client code can run visitor operations over any set of elements without
    figuring out their concrete classes. The accept operation directs a call to
    the appropriate operation in the visitor object.
    """

    # ...
    for component in components:
        component.accept(visitor)
    # ...

components = [ConcreteComponentA(), ConcreteComponentB()]

print("The client code works with all visitors via the base Visitor interface:")
visitor1 = ConcreteVisitor1()
client_code(components, visitor1)

print("It allows the same client code to work with different types of visitors:")
visitor2 = ConcreteVisitor2()
client_code(components, visitor2)

The client code works with all visitors via the base Visitor interface:
A + ConcreteVisitor1
B + ConcreteVisitor1
It allows the same client code to work with different types of visitors:
A + ConcreteVisitor2
B + ConcreteVisitor2


In [72]:
from pyleri import (
    Choice,
    Grammar,
    Keyword,
    List,
    Optional,
    Regex,
    Repeat,
    Ref,
    Sequence)

import json
import schemdraw
import schemdraw.elements as elm
from collections import defaultdict

class CircuitJSGrammar(Grammar):
    START = Ref()
    
    number_literal = Regex('[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?')
    two_terminal_coords = Repeat(number_literal, mi=4, ma=4)
    booly = Choice(Keyword('true'), Keyword('false'))
    string_literal = Regex('[A-Za-z0-9_]+') 

    capacitor = Sequence(Keyword('c'), two_terminal_coords, number_literal, number_literal, Repeat(number_literal, mi=2, ma=2))
    ground = Sequence(Keyword('g'), two_terminal_coords, Repeat(number_literal, mi=2, ma=2))    
    inductor = Sequence(Keyword('l'),  two_terminal_coords, number_literal, number_literal, Repeat(number_literal, mi=2, ma=2))
    npntransistor = Sequence(Keyword('t'), two_terminal_coords, Repeat(number_literal, mi=5, ma=5), string_literal)
    pchannelmosfet = Sequence(Keyword('f'), two_terminal_coords, Repeat(number_literal, mi=4, ma=4))
    resistor = Sequence(Keyword('r'), two_terminal_coords, number_literal, number_literal)
    switch = Sequence(Keyword('s'), two_terminal_coords, number_literal, number_literal, booly)
    voltage = Sequence(Keyword('v'), two_terminal_coords, Repeat(number_literal, mi=3, ma=3), number_literal, Repeat(number_literal, mi=2, ma=2), number_literal)
    wire = Sequence(Keyword('w'), two_terminal_coords, number_literal)

    START = Choice(capacitor, ground, inductor, npntransistor, pchannelmosfet, resistor, switch, voltage, wire)
   
def node_props(node, children):
    return {
        'start': node.start,
        'end': node.end,
        'name': node.element.name if hasattr(node.element, 'name') else None,
        'element': node.element.__class__.__name__,
        'string': node.string,
        'children': children
    }


# Recursive method to get the children of a node object:
def get_children(children):
    return [node_props(c, get_children(c.children)) for c in children]


# View the parse tree:
def view_parse_tree(res):
    start = res.tree.children[0] \
        if res.tree.children else res.tree
    return node_props(start, get_children(start.children))
        
grammar = CircuitJSGrammar()
#print(grammar.parse('r 240 80 448 80 0 10').is_valid)
#res = grammar.parse('r 240 80 448 80 0 10')
#print(json.dumps(view_parse_tree(res), indent=2))
#print(grammar.parse('v 32 288 32 224 0 0 40 12 0 0 0.5').is_valid)

#res = grammar.parse('c 368 64 368 144 0 1e-7 0.001 0.001')
#print(res.is_valid)
#print(res.pos)


In [73]:
class ComponentWarehouse:
    def __init__(self):
        self.classes = {}

    def add_class(self, c):
        self.classes[c.__name__] = c

    # -- the decorator
    def component(self, c):
        self.add_class(c)

        # Decorators have to return the function/class passed (or a modified variant thereof), however I'd rather do this separately than retroactively change add_class, so.
        # "held" is more succint, anyway.
        return c 

    def __getitem__(self, n):
        return self.classes[n]

component_warehouse = ComponentWarehouse()

In [74]:
class ElectricComponent(object):
    
    id = 1

    def __init__(self, start_coords, end_coords, terminal_start_coord, terminal_end_coord):
        self._start_coords = start_coords
        self._end_coords = end_coords
        self._terminal_start_coords = terminal_start_coord
        self._terminal_end_coords = terminal_end_coord

    @property
    def start_coords(self):
        return self._start_coords

    @property
    def end_coords(self):
        return self._end_coords

    @property
    def hasValue(self):
        return True

    @property
    def hasLabel(self):
        return True

    @property
    def sorted_endpoints(self):
        terminal_coords = [self._start_coords, self._end_coords]
        sorted_coordinates = sorted(terminal_coords, key=lambda x: (x[0], int(x[1])))
        return sorted_coordinates

    def _direction(self):
        x1, y1 = self._start_coords
        x2, y2 = self._end_coords
        diff_x = x1 - x2
        diff_y = y1 - y2

        match (diff_x, diff_y):
            case (0, diff_y) if diff_y < 0:
                return "down"
            case (0, diff_y) if diff_y > 0:
                return "up"
            case (diff_x, 0) if diff_x < 0:
                return "right"
            case (diff_x, 0) if diff_x > 0:
                return "left"
            case _:
                return "up"   

    def _direction_terminal(self):
        x1, y1 = self._terminal_start_coords
        x2, y2 = self._terminal_end_coords
        diff_x = x1 - x2
        diff_y = y1 - y2

        match (diff_x, diff_y):
            case (0, diff_y) if diff_y < 0:
                return "down"
            case (0, diff_y) if diff_y > 0:
                return "up"
            case (diff_x, 0) if diff_x < 0:
                return "right"
            case (diff_x, 0) if diff_x > 0:
                return "left"
            case _:
                return "up"   


    def _direction_original(self):
        x1, y1 = self._start_coords
        x2, y2 = self._end_coords
        diff_x = x1 - x2
        diff_y = y1 - y2

        match (diff_x, diff_y):
            case (0, diff_y) if diff_y < 0:
                return "down"
            case (0, diff_y) if diff_y > 0:
                return "up"
            case (diff_x, 0) if diff_x < 0:
                return "right"
            case (diff_x, 0) if diff_x > 0:
                return "left"
            case _:
                return "up"          

    @property
    def direction(self):
        return self._direction()      

    @property
    def shouldReverse(self):
        return False  

    def accept(self, d, v):
        return v.visit_any(d, self)

    

In [75]:
class Visitor(ABC):
    @abstractmethod
    def visit_any(self, d, element):
        pass

    @abstractmethod
    def visit_three_terminal(self, d, element):
        pass

In [76]:

class TwoTerminalComponent(ElectricComponent):
    def setValue(self, parsing_element):
        self._value = parsing_element[3].string
        print(self._value)
        
class TwoTerminalDirectionalComponent(TwoTerminalComponent):
    @property
    def shouldReverse(self):
        direction = self._direction_terminal()
        match direction:
            case 'up'|'left':
                return True
            case _:
                return False

class ThreeTerminalComponent(ElectricComponent):
    pass


In [77]:
@component_warehouse.component
class capacitor(TwoTerminalComponent):
    class Units(Enum):
        farads = "F"
        microfarads = "µF"
        picofarads = "pF"

    def getElement(self):
        return elm.Capacitor

    @property
    def labelPrefix(self):
        return "C"

@component_warehouse.component
class ground(ElectricComponent):
    def getElement(self):
        return elm.Ground

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False

    @property
    def direction(self):
        element_direction = self._direction()
        match element_direction:
            case 'down':
                return 'right'
            case _:
                return element_direction

@component_warehouse.component
class inductor(TwoTerminalComponent):
    class Units(Enum):
        henry = "H"
        millihenry = "mH"
        microhenry = "µH"
    
    def getElement(self):
        return elm.Inductor

@component_warehouse.component
class npntransistor(TwoTerminalComponent):
    def getElement(self):
        return elm.transistors.BjtNpn

@component_warehouse.component
class pchannelmosfet(TwoTerminalComponent):
    def getElement(self):
        return elm.transistors.PFet

@component_warehouse.component
class resistor(TwoTerminalComponent):
    class Units(Enum):
        ohms = "Ω"
        killohms = "kΩ"
        megaohms = "mΩ"

    def getElement(self):
        return elm.Resistor

    @property
    def labelPrefix(self):
        return "R"

@component_warehouse.component
class switch(TwoTerminalComponent):
    def getElement(self):
        return elm.Switch

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False    

@component_warehouse.component
class voltage(TwoTerminalDirectionalComponent):
    class Units(Enum):
        volts = "V"

    def getElement(self):
        return elm.SourceV

    @property
    def labelPrefix(self):
        return "V"

    @property
    def isDirectional(self):
        return True

@component_warehouse.component
class wire(ElectricComponent):
    def getElement(self):
        return elm.Line

    @property
    def hasValue(self):
        return False

    @property
    def hasLabel(self):
        return False




In [78]:
class SchemDrawVisitor(Visitor):    
    def visit_any(self, d, element):
        d.push()
        element_args = {}
        element_args['d'] = element.direction
        if element.shouldReverse:
            element_args["reverse"] = True
        c = element.getElement()(**element_args)
        d += c
        here = d.here
        d.pop()
        return here

    def visit_three_terminal(self, d, element):
        pass

In [79]:
def getCoordinates(parsing_element):
    coords = parsing_element.split()
    return [(int(coords[0]), int(coords[1])), (int(coords[2]), int(coords[3]))]

In [80]:
def parse_component(parsing_result):
    start = parsing_result.tree.children[0].children[0]
    component_name = start.element.name
    coordinates = start.children[1].string
    terminal_coords = getCoordinates(coordinates)
    sorted_coordinates = sorted(terminal_coords, key=lambda x: (int(x[0]), int(x[1])))
    start_coords, end_coords = sorted_coordinates
    return {
        "component_name": component_name,
        "terminal_coords": terminal_coords,
        "sorted_coords": sorted_coordinates,
        "start_coords": start_coords,
        "end_coords": end_coords,
    }

In [81]:
def find_left_corner_most(coords_list):
    leftcorner_most = coords_list[0]
    for coord in coords_list[1:]:
        boo = [leftcorner_most, coord]
        sorted_boo = sorted(boo, key=lambda x: (x[0], x[1]))
        if boo != sorted_boo:
            leftcorner_most = coord
    return leftcorner_most   

def find_left_corner_most_from_drawn_list(coords_list, drawn_coords):
    leftcorner_most = coords_list[0]
    for coord in coords_list[1:]:
        boo = [leftcorner_most, coord]
        sorted_boo = sorted(boo, key=lambda x: (x[0], x[1]))
        if boo != sorted_boo and coord in drawn_coords:
            leftcorner_most = coord
    return leftcorner_most          



In [82]:
def create_component(component_manifest, component_warehouse):
    electric_component_class = component_warehouse[component_manifest['component_name']]
    x,y = component_manifest['terminal_coords']
    return electric_component_class(component_manifest['start_coords'], component_manifest['end_coords'], x, y)
    


In [83]:
def update_lookup(lookup, component):
    lookup[component["start_coords"]].append((component, "start"))
    lookup[component["end_coords"]].append((component, "end"))
    return lookup

In [84]:
#input_file = "tests/test005.txt"
#output_file = "out.svg"

def main(input_file, output_file):
    schemdraw.use('svg')
    schemdraw.svgconfig.text = 'path'
    schemdraw.svgconfig.svg2 = False
    schemdraw.svgconfig.precision = 2

    visitor = SchemDrawVisitor()

    if __name__ == "__main__":
        grammar = CircuitJSGrammar()

        lookup = defaultdict(list)
        elements_to_draw = []
        elements_drawn = []
        candidate_coords = []
        drawn_list = {}
        number_of_elements = 0

        with open(input_file, "r") as f:
            f.readline()
            for line in f:            
                parsing_result = grammar.parse(line)
                #print(parsing_result.is_valid)
                if parsing_result.is_valid:
                    component = parse_component(parsing_result)
                    lookup = update_lookup(lookup, component)
                    number_of_elements = number_of_elements + 1

            candidate_coords = list(set(list((lookup.keys()))))         
            leftcorner_most_coord = find_left_corner_most(candidate_coords)        
            #print(number_of_elements)
        

        with schemdraw.Drawing(show=False, file=output_file) as d:
            while not len(elements_drawn) == number_of_elements:
            #for _ in range(5):
                components_to_draw = lookup[leftcorner_most_coord]
                for component_lookup in components_to_draw:                
                    component_manifest, anchor = component_lookup
                    if not anchor == 'end':
                        component = create_component(component_manifest, component_warehouse)            
                        endpoint = component.accept(d, visitor)
                        x,y =endpoint
                        x = round(x,1)
                        y = round(y,1)
                        #print(component_manifest)
                        drawn_list[component_manifest['end_coords']] = [x,y]
                        elements_drawn.append(component_lookup)
                print(leftcorner_most_coord)
                #print(candidate_coords)
                candidate_coords.remove(leftcorner_most_coord)
                # todo - should also be in the drawn_list also
                #leftcorner_most_coord = find_left_corner_most(candidate_coords)
                print(leftcorner_most_coord)
                print(drawn_list)
                leftcorner_most_coord = find_left_corner_most_from_drawn_list(candidate_coords, drawn_list.keys())
                d.here = drawn_list[leftcorner_most_coord]
                d.theta = 0
                #print(drawn_list)
                #print(leftcorner_most_coord)

'''
max_test_number = 5
for i in range(max_test_number):
    num = str(i+1).zfill(3) 
    input_file = f"tests/test{num}.txt"
    output_file = f"out/test{num}.svg"
    main(input_file, output_file)        
'''

main("tests/test006.txt", "out/test006.svg")


                    


(96, 208)
(96, 208)
{(96, 336): [-0.0, -3.0], (272, 208): [3.0, 0.0]}
(96, 336)
(96, 336)
{(96, 336): [-0.0, -3.0], (272, 208): [3.0, 0.0], (368, 336): [3.0, -3.0]}
(272, 208)
(272, 208)
{(96, 336): [-0.0, -3.0], (272, 208): [3.0, 0.0], (368, 336): [3.0, -3.0], (368, 208): [3.8, 0.7]}


KeyError: (368, 80)